# Аналіз дадзеных імёнаў у ОСМ Беларусі

## Зьмест

- Праблематыка
- Спампуем дамп ОСМ
- Усталёўваем залежанасьцьі
- Пошук сувязей дзеля падтрыманьня спасылачнай цэласнасьці

## Праблематыка

У беларускім ОСМ шырока выкарыстоўваюцца беларуская і расейская мова, для іх ёсьць адпаведнікі `name:be` і `name:ru`, таксама мовы выкарыстоўваюцца ў агульных тэгах як `name`, `addr:*` і іншых. Праблематка выкарыстоўваньня аднае, ці іншае, ці абедзьвух моваў апісанае тут https://wiki.openstreetmap.org/wiki/BE:Belarus_language_issues. Незалежна ад варыянту выкарыстоўваньня мовы павінны вытрымлівацца наступныя правілы: пошук на любое мове мусіць працаваць, павінна быць магчымасьць паказываць подпісы на любой мове (ці ў арыгінале, але гэтае правіла зараз не выконваецца), павінна захоўвацца спасылкавая цэласнасьць (што можа ўплываць на папярэднія два пункты).

Гэты аналіз ставіць мэтаю знайсьці адпаведныя катэгорыі і тэгі ў якіх кірылічныя значэньні на беларускай альбо расейскай мовах выкарыстоўваюцца як спасылка.

## Спампуем дамп ОСМ

In [37]:
!wget https://download.geofabrik.de/europe/belarus-latest.osm.pbf

--2021-03-17 18:35:46--  https://download.geofabrik.de/europe/belarus-latest.osm.pbf
Resolving download.geofabrik.de (download.geofabrik.de)... 88.99.142.44, 138.201.219.183, 116.202.112.212
Connecting to download.geofabrik.de (download.geofabrik.de)|88.99.142.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 242289054 (231M) [application/octet-stream]
Saving to: ‘belarus-latest.osm.pbf.1’

belarus-latest.osm. 100%[===================>] 231.06M  1.15MB/s    in 2m 26s  

2021-03-17 18:38:13 (1.58 MB/s) - ‘belarus-latest.osm.pbf.1’ saved [242289054/242289054]



## Усталюем залежнасьці

In [39]:
!pip install https://github.com/lechup/imposm-parser/archive/python3.zip

/bin/bash: pip: command not found


## Пошук залезнасьцяў

In [1]:
from collections import defaultdict, Counter

from imposm.parser import OSMParser


cirylic_chars = frozenset('абвгдеёжзіийклмнопрстуўфхцчшщьыъэюяАБВГДЕЁЖЗІИІЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ')

# this keys should not be used as links
skip_lang_names = {
    'name:left',
    'name:right',
    'int_name',
    'loc_name',
    'nat_name',
    'old_name',
    'reg_name',
    'sorting_name',
    'alt_name',
}

# language specific keys prefixes also should not be used as links
skip_lang_names_prefixes = (
    'name:',
    'loc_name:',
    'nat_name:',
    'official_name:',
    'old_name:',
    'reg_name:',
    'short_name:',
    'sorting_name:',
    'alt_name:',
)

# also skip keys with most common lang suffixes
skip_lang_suffixes = (
    ':be',
    ':ru',
)

In [2]:
key_counter = Counter()
key_val_counter = defaultdict(Counter)
key_val_tag_counter = defaultdict(lambda: defaultdict(Counter))


def process(params):
    for _, tags, _ in params:
        keys_and_tags = set()
        for k, v in tags.items():
            if k.startswith(skip_lang_names_prefixes):
                continue
            if k.endswith(skip_lang_suffixes):
                continue
            keys_and_tags |= {(k,)} | {(k, v)}
        for k, v in tags.items():
            if frozenset(v) & cirylic_chars:
                key_counter[k] += 1
            if k in skip_lang_names:
                continue
            if k.startswith(skip_lang_names_prefixes):
                continue
            if k.endswith(skip_lang_suffixes):
                continue
            if len(frozenset(v) & cirylic_chars) > 0:
                key_val_counter[k][v] += 1
                for key_or_tag in keys_and_tags - {(k,)} - {(k, v)}:
                    key_val_tag_counter[k][v][key_or_tag] += 1

                        
OSMParser(
    nodes_callback=process,
    ways_callback=process,
    relations_callback=process,
).parse('belarus-latest.osm.pbf')

In [3]:
# cirylic tag values count
for k, c in key_counter.most_common():
    uniq_values = len(key_val_counter[k]) or ''
    print(f'{k.ljust(30)} {str(c).rjust(6)} {str(uniq_values).rjust(6)}')

addr:street                    813801   9562
name                           314939 109482
name:ru                        203692       
name:be                        186995       
addr:city                       81913   1950
addr:housenumber                72933   5390
ref                             64029  11856
name:prefix                     57217       
addr:district                   51156    144
addr:region                     50941     13
addr:place                      35625   1151
operator                        34810   6060
wikipedia                       33095  29299
official_name                   24827   8400
name:prefix:ru                  12569       
description                     11916   5886
addr:subdistrict                10563    767
source:population               10348      1
alt_name:be                      8545       
name:be-tarask                   8225       
minsk_PT:note                    6445      3
source                           5489     77
name:uk   

In [8]:
eq_keys_pair_vals1_counter = Counter()
eq_keys_pair_vals2_counter = Counter()
eq_keys_pair_tag_vals1_counter = defaultdict(Counter)
eq_keys_pair_tag_vals2_counter = defaultdict(Counter)
eq_keys_pair_val_tag_vals1_counter = defaultdict(Counter)
eq_keys_pair_val_tag_vals2_counter = defaultdict(Counter)

in_keys_pair_vals1_counter = Counter()
in_keys_pair_vals2_counter = Counter()
in_keys_pair_tag_vals1_counter = defaultdict(Counter)
in_keys_pair_tag_vals2_counter = defaultdict(Counter)
in_keys_pair_val_tag_vals1_counter = defaultdict(Counter)
in_keys_pair_val_tag_vals2_counter = defaultdict(Counter)


for k1, vals_counter1 in sorted(key_val_counter.items(), key=lambda kv: -len(kv[1])):
    for k2, vals_counter2 in sorted(key_val_counter.items(), key=lambda kv: -len(kv[1])):
        if k1 == k2:
            continue

        x = (k1, k2)
        for val1, vals_count1 in vals_counter1.items():    
            for val2, vals_count2 in vals_counter2.items():
                if val1 == val2:
                    eq_keys_pair_vals1_counter[x] += vals_count1
                    eq_keys_pair_vals2_counter[x] += vals_count2
                    
                    eq_keys_pair_tag_vals1_counter[x][val1] += vals_count1
                    eq_keys_pair_tag_vals2_counter[x][val2] += vals_count2
                    
                    for key_or_tag, tags_count1 in key_val_tag_counter[k1][val1].items():
                        eq_keys_pair_val_tag_vals1_counter[x][key_or_tag] += tags_count1
                    for key_or_tag, tags_count2 in key_val_tag_counter[k2][val2].items():
                        eq_keys_pair_val_tag_vals2_counter[x][key_or_tag] += tags_count2
                if val1 in val2:                    
                    in_keys_pair_vals1_counter[x] += vals_count1
                    in_keys_pair_vals2_counter[x] += vals_count2
                
                    in_keys_pair_tag_vals1_counter[x][val1] += vals_count1
                    in_keys_pair_tag_vals2_counter[x][val2] += vals_count2
                    
                    for key_or_tag, tags_count1 in key_val_tag_counter[k1][val1].items():
                        in_keys_pair_val_tag_vals1_counter[x][key_or_tag] += tags_count1
                    for key_or_tag, tags_count2 in key_val_tag_counter[k2][val2].items():
                        in_keys_pair_val_tag_vals2_counter[x][key_or_tag] += tags_count2


In [14]:
in_keys_pair_vals1_counter

Counter({('name', 'wikipedia'): 7120209,
         ('name', 'ref'): 3606366,
         ('name', 'addr:street'): 2100218,
         ('name', 'official_name'): 2615396,
         ('name', 'operator'): 1342987,
         ('name', 'description'): 1935116,
         ('name', 'addr:housenumber'): 89867,
         ('name', 'addr:city'): 311035,
         ('name', 'short_name'): 326306,
         ('name', 'note'): 530900,
         ('name', 'addr:place'): 199146,
         ('name', 'addr:subdistrict'): 125655,
         ('name', 'destination'): 159433,
         ('name', 'inscription'): 237596,
         ('name', 'addr:housename'): 88470,
         ('name', 'to'): 77934,
         ('name', 'from'): 76185,
         ('name', 'ref:mcrb'): 799,
         ('name', 'fixme'): 94379,
         ('name', 'addr2:street'): 74297,
         ('name', 'fire_hydrant:street'): 45430,
         ('name', 'ref:minskgrado'): 52,
         ('name', 'brand'): 45955,
         ('name', 'fire_hydrant:housenumber'): 16865,
         ('name',

In [16]:
def sortv(counter):
    return sorted(counter.items(), key=lambda kv: -kv[1])


shown = set()
keys = Counter()
for k in sorted(in_keys_pair_vals1_counter.keys(), 
                key=lambda k: -max(in_keys_pair_vals1_counter[k], in_keys_pair_vals2_counter[k])):
    x = tuple(sorted(k))
    if x in shown:
        continue
    shown.add(x)
    keys[k[0]] += 1
    keys[k[1]] += 1
    
    print(*k, in_keys_pair_vals1_counter[k], in_keys_pair_vals2_counter[k], 
              eq_keys_pair_vals1_counter[k], eq_keys_pair_vals2_counter[k])
    print('\t', len(in_keys_pair_tag_vals1_counter[k]), *sortv(in_keys_pair_tag_vals1_counter[k])[:3])
    print('\t', len(in_keys_pair_tag_vals2_counter[k]), *sortv(in_keys_pair_tag_vals2_counter[k])[:3])
    print('\t', len(in_keys_pair_val_tag_vals1_counter[k]), *sortv(in_keys_pair_val_tag_vals1_counter[k])[:3])
    print('\t', len(in_keys_pair_val_tag_vals2_counter[k]), *sortv(in_keys_pair_val_tag_vals1_counter[k])[:3])
    print('\t', len(eq_keys_pair_tag_vals1_counter[k]), *sortv(eq_keys_pair_tag_vals1_counter[k])[:3])
    print('\t', len(eq_keys_pair_tag_vals2_counter[k]), *sortv(eq_keys_pair_tag_vals2_counter[k])[:3])
    print('\t', len(eq_keys_pair_val_tag_vals1_counter[k]), *sortv(eq_keys_pair_val_tag_vals1_counter[k])[:3])
    print('\t', len(eq_keys_pair_val_tag_vals2_counter[k]), *sortv(eq_keys_pair_val_tag_vals2_counter[k])[:3])


addr:housenumber name 17254739 431507 40536 4912
	 393 ('н', 15592518) ('Н', 571680) ('ж', 384454)
	 85227 ('Молодёжная улица', 6688) ('Центральная улица', 5298) ('Лесная улица', 2464)
	 15009 (('building',), 17233859) (('building', 'yes'), 16723077) (('addr:street',), 14553334)
	 139940 (('building',), 17233859) (('building', 'yes'), 16723077) (('addr:street',), 14553334)
	 207 ('1А', 3265) ('2А', 3210) ('3А', 1618)
	 207 ('Магазин', 534) ('Н', 382) ('КН', 369)
	 13584 (('building',), 39164) (('addr:street',), 38631) (('building', 'yes'), 23577)
	 1674 (('building',), 4025) (('building', 'yes'), 2533) (('addr:street',), 990)
addr:housename name 12097466 823503 1278 10788
	 326 ('н', 9901818) ('Н', 1329156) ('к', 390194)
	 102808 ('Центральная улица', 7947) ('Молодёжная улица', 6688) ('Советская улица', 5970)
	 770 (('building',), 12076432) (('building', 'yes'), 11757554) (('addr:street',), 8858907)
	 173658 (('building',), 12076432) (('building', 'yes'), 11757554) (('addr:street',), 8

	 9097 (('addr:region',), 1009306) (('addr:country',), 997173) (('addr:country', 'BY'), 992220)
	 0
	 0
	 0
	 0
addr:housename description 998590 49204 866 263
	 129 ('н', 800574) ('Н', 111879) ('к', 27692)
	 5805 ('Минская Кольцевая АвтоДорога', 4221) ('Шиномонтаж', 1790) ('Аптечная сеть, занимающаяся розничной торговлей товарами для красоты и здоровья', 624)
	 474 (('building',), 996984) (('building', 'yes'), 971739) (('addr:street',), 734567)
	 16519 (('building',), 996984) (('building', 'yes'), 971739) (('addr:street',), 734567)
	 45 ('Н', 279) ('н', 174) ('кн', 99)
	 45 ('н/ж', 53) ('н', 53) ('кн', 46)
	 284 (('building',), 856) (('building', 'yes'), 829) (('addr:street',), 694)
	 194 (('building',), 252) (('building', 'yes'), 216) (('addr:street',), 169)
addr:housenumber operator 950112 60643 33 29
	 67 ('н', 832412) ('Н', 48240) ('П', 34035)
	 4944 ('ГП "Столичный транспорт и связь"', 15664) ('КТУП «Гомельоблпассажиртранс»', 4995) ('Минские кабельные сети', 1278)
	 5146 (('build

	 22899 (('addr:street',), 243205) (('addr:housenumber',), 240551) (('building',), 195251)
	 1159 (('addr:street',), 243205) (('addr:housenumber',), 240551) (('building',), 195251)
	 16 ('СТ "Мелиоратор"', 94) ('СТ "Берёзка УВД"', 85) ('СТ "Энергетик-1"', 50)
	 16 ('СТ "Колос"', 9) ('СТ "Мелиоратор"', 9) ('СТ "Энергетик-1"', 4)
	 312 (('building',), 322) (('addr:housenumber',), 308) (('building', 'house'), 286)
	 55 (('name',), 38) (('official_name',), 38) (('official_status', 'садоводческое товарищество'), 36)
fire_hydrant:city wikipedia 245981 909 0 0
	 4 ('Минск', 243840) ('Речица', 1532) ('Могилёв', 576)
	 819 ('ru:Улица Есенина (Минск)', 22) ('ru:Улица Маяковского (Минск)', 12) ('ru:Красноармейская улица (Минск)', 7)
	 1639 (('fire_hydrant:type',), 245981) (('emergency', 'fire_hydrant'), 245981) (('emergency',), 245981)
	 3190 (('fire_hydrant:type',), 245981) (('emergency', 'fire_hydrant'), 245981) (('emergency',), 245981)
	 0
	 0
	 0
	 0
addr:housenumber addr:city 238825 80922 82

	 1651 (('building',), 65385) (('building', 'yes'), 55972) (('building', 'residential'), 7327)
	 269 ('Молодёжная улица', 1672) ('улица Победы', 311) ('Евроопт', 167)
	 269 ('Вокзал', 42) ('Гомель', 34) ('Фолюш', 13)
	 2915 (('highway',), 2960) (('int_name',), 2728) (('highway', 'residential'), 1650)
	 1284 (('route',), 600) (('name',), 600) (('type',), 599)
name from 76185 4561 5294 596
	 582 ('н', 51245) ('Н', 3438) ('к', 1720)
	 368 ('Вокзал', 220) ('ДС Малиновка-4', 120) ('Гомель', 108)
	 5456 (('building',), 63476) (('building', 'yes'), 54916) (('building', 'residential'), 6546)
	 1636 (('building',), 63476) (('building', 'yes'), 54916) (('building', 'residential'), 6546)
	 259 ('Молодёжная улица', 1672) ('улица Победы', 311) ('Белинвестбанк', 222)
	 259 ('Вокзал', 44) ('Гомель', 36) ('Фолюш', 13)
	 3149 (('highway',), 2950) (('int_name',), 2904) (('highway', 'residential'), 1650)
	 1289 (('name',), 595) (('to',), 594) (('type',), 594)
addr:city full_name 75834 26 0 0
	 15 ('Гродн

	 0
	 0
	 0
	 0
destination wikipedia 40849 5372 0 0
	 171 ('Гомель', 18016) ('Мінск', 7839) ('Магілёў', 3213)
	 4418 ('be:Праспект Дзяржынскага (Мінск)', 202) ('be:Сож', 51) ('be:Лоша (прыток Ашмянкі)', 25)
	 609 (('highway',), 36509) (('oneway', 'yes'), 36507) (('oneway',), 36507)
	 12942 (('highway',), 36509) (('oneway', 'yes'), 36507) (('oneway',), 36507)
	 0
	 0
	 0
	 0
addr:city to 39889 185 12796 89
	 73 ('Минск', 28140) ('Гродно', 5870) ('Гомель', 1172)
	 84 ('Гомель', 34) ('ДС Малиновка-4', 16) ('Октябрьский', 4)
	 22036 (('addr:street',), 38232) (('addr:housenumber',), 37263) (('building',), 29080)
	 505 (('addr:street',), 38232) (('addr:housenumber',), 37263) (('building',), 29080)
	 38 ('Минск', 9380) ('Гомель', 586) ('Тюрли', 408)
	 38 ('Гомель', 34) ('Новополоцк', 3) ('Полоцк', 3)
	 14852 (('addr:street',), 12153) (('addr:housenumber',), 11609) (('building',), 9444)
	 257 (('route',), 89) (('name',), 89) (('type',), 88)
description ref 19940 39010 68 780
	 41 ('н', 10494)

...

	 0
	 0
ref source_2 1 1 0 0
	 1 ('Г', 1)
	 1 ('Грузоперевозки', 1)
	 8 (('name', 'Зал Г'), 1) (('indoor', 'area'), 1) (('indoor',), 1)
	 18 (('name', 'Зал Г'), 1) (('indoor', 'area'), 1) (('indoor',), 1)
	 0
	 0
	 0
	 0
ref lamp_type 1 1 0 0
	 1 ('Р', 1)
	 1 ('electricРезультаты_поиска_Studia_-_uczelnia_|_Szkoła_Główna_Handlowa', 1)
	 2 (('aeroway', 'taxiway'), 1) (('aeroway',), 1)
	 2 (('aeroway', 'taxiway'), 1) (('aeroway',), 1)
	 0
	 0
	 0
	 0
ref captured:with 1 1 0 0
	 1 ('Г', 1)
	 1 ('Галина Шкрадюк', 1)
	 8 (('name', 'Зал Г'), 1) (('indoor', 'area'), 1) (('indoor',), 1)
	 6 (('name', 'Зал Г'), 1) (('indoor', 'area'), 1) (('indoor',), 1)
	 0
	 0
	 0
	 0
ref library 1 1 0 0
	 1 ('Г', 1)
	 1 ('Городская библиотека № 5', 1)
	 8 (('name', 'Зал Г'), 1) (('indoor', 'area'), 1) (('indoor',), 1)
	 4 (('name', 'Зал Г'), 1) (('indoor', 'area'), 1) (('indoor',), 1)
	 0
	 0
	 0
	 0
ref opening_hours:covid19 1 1 0 0
	 1 ('Р', 1)
	 1 ('Mo-Su 07:00-24:00 open "Ресторан, McDrive"|| Mo-Su 07:00-

	 1 ('Внутри двухъярусный иконостас из 17 икон, икона 18в. «Богоматерь», деревянная скульптура 19в. «Распятие», посветлели краски на иконе (обновляется сама собой) «Святого Феодосия Черниговского» Храм никогда не закрывался', 1)
	 14 (('maxspeed', '60'), 1) (('destination:int_ref',), 1) (('maxaxleload',), 1)
	 18 (('maxspeed', '60'), 1) (('destination:int_ref',), 1) (('maxaxleload',), 1)
	 0
	 0
	 0
	 0
destination seats 1 1 0 0
	 1 ('Серебрянка', 1)
	 1 ('Д/С Серебрянка', 1)
	 4 (('highway',), 1) (('maxspeed',), 1) (('highway', 'service'), 1)
	 12 (('highway',), 1) (('maxspeed',), 1) (('highway', 'service'), 1)
	 0
	 0
	 0
	 0
inscription addr2:street 1 1 0 0
	 1 ('Мир', 1)
	 1 ('Мирная улица', 1)
	 12 (('note',), 1) (('material', 'steel'), 1) (('wikidata', 'Q11115124'), 1)
	 10 (('note',), 1) (('material', 'steel'), 1) (('wikidata', 'Q11115124'), 1)
	 0
	 0
	 0
	 0
inscription subject:wikipedia 1 1 0 0
	 1 ('Мир', 1)
	 1 ('ru:Киров, Сергей Миронович', 1)
	 12 (('note',), 1) (('materi

	 0
designation email 1 1 0 0
	 1 ('б', 1)
	 1 ('6684126@.бу', 1)
	 8 (('name', 'ООО"БЕЛ-ИЗОЛИТ-СЕРВИС"'), 1) (('addr:street',), 1) (('building', 'yes'), 1)
	 8 (('name', 'ООО"БЕЛ-ИЗОЛИТ-СЕРВИС"'), 1) (('addr:street',), 1) (('building', 'yes'), 1)
	 0
	 0
	 0
	 0
designation description:uk 1 1 0 0
	 1 ('б', 1)
	 1 ('Можна розбити намет над озером', 1)
	 8 (('name', 'ООО"БЕЛ-ИЗОЛИТ-СЕРВИС"'), 1) (('addr:street',), 1) (('building', 'yes'), 1)
	 10 (('name', 'ООО"БЕЛ-ИЗОЛИТ-СЕРВИС"'), 1) (('addr:street',), 1) (('building', 'yes'), 1)
	 0
	 0
	 0
	 0
designation natural 1 1 0 0
	 1 ('б', 1)
	 1 ('Прылепская сельская бiблiятэка', 1)
	 8 (('name', 'ООО"БЕЛ-ИЗОЛИТ-СЕРВИС"'), 1) (('addr:street',), 1) (('building', 'yes'), 1)
	 6 (('name', 'ООО"БЕЛ-ИЗОЛИТ-СЕРВИС"'), 1) (('addr:street',), 1) (('building', 'yes'), 1)
	 0
	 0
	 0
	 0
designation old_loc_name 1 1 0 0
	 1 ('б', 1)
	 1 ('Бобровый остров', 1)
	 8 (('name', 'ООО"БЕЛ-ИЗОЛИТ-СЕРВИС"'), 1) (('addr:street',), 1) (('building', 'yes'), 1)
	 

addr:unit addr:housenumber_1 1 1 0 0
	 1 ('А', 1)
	 1 ('18А', 1)
	 8 (('addr:street',), 1) (('addr:housenumber', '80'), 1) (('building',), 1)
	 8 (('addr:street',), 1) (('addr:housenumber', '80'), 1) (('building',), 1)
	 0
	 0
	 0
	 0
addr:unit old:addr:housenumber 1 1 0 0
	 1 ('А', 1)
	 1 ('14А', 1)
	 8 (('addr:street',), 1) (('addr:housenumber', '80'), 1) (('building',), 1)
	 26 (('addr:street',), 1) (('addr:housenumber', '80'), 1) (('building',), 1)
	 0
	 0
	 0
	 0
addr:unit name_official 1 1 0 0
	 1 ('Б', 1)
	 1 ('Церковь Покрова Пресвятой Богородицы', 1)
	 18 (('addr:housenumber', '44'), 1) (('surface',), 1) (('amenity', 'parking'), 1)
	 18 (('addr:housenumber', '44'), 1) (('surface',), 1) (('amenity', 'parking'), 1)
	 0
	 0
	 0
	 0
addr:unit description2 1 1 0 0
	 1 ('Б', 1)
	 1 ('Внутри двухъярусный иконостас из 17 икон, икона 18в. «Богоматерь», деревянная скульптура 19в. «Распятие», посветлели краски на иконе (обновляется сама собой) «Святого Феодосия Черниговского» Храм никогд

memorial office 1 1 0 0
	 1 ('мемориал', 1)
	 1 ('Подъезд от а/д Н8572 к мемориальному комплексу "Мосевичи"', 1)
	 6 (('material',), 1) (('historic', 'memorial'), 1) (('name',), 1)
	 12 (('material',), 1) (('historic', 'memorial'), 1) (('name',), 1)
	 0
	 0
	 0
	 0
destination:street:backward destination 1 1 0 0
	 1 ('вуліца Убарэвіча', 1)
	 1 ('вуліца Убарэвіча;вуліца Ташкенцкая', 1)
	 16 (('surface',), 1) (('bicycle',), 1) (('maxspeed',), 1)
	 12 (('surface',), 1) (('bicycle',), 1) (('maxspeed',), 1)
	 0
	 0
	 0
	 0
destination:street:backward destination:street:forward 1 1 0 0
	 1 ('вуліца Убарэвіча', 1)
	 1 ('вуліца Убарэвіча;вуліца Ташкенцкая', 1)
	 16 (('surface',), 1) (('bicycle',), 1) (('maxspeed',), 1)
	 22 (('surface',), 1) (('bicycle',), 1) (('maxspeed',), 1)
	 0
	 0
	 0
	 0
destination:ref:backward fixme 1 1 0 0
	 1 ('М14', 1)
	 1 ('Вся трасса М14 доступна для езды на велосипеде в соответствии с ПДД РБ. Прошу исправить данный момент', 1)
	 16 (('surface',), 1) (('bicycle',)

## Вынікі сувязей спасылкавай цэласнасьці

### Пазначэньні
- `==` - супадзеньне значэньняў
- `IN` - першае значэньне выкарыстоўваецца ў другім
- `key` - ключ
- `key=val` - тэг
- `key1 + key2` - спалучэньне тэгаў
- `key1 + key2 == key3 + key4` - пры спалучэньні ключэй `key1 + key2` і `key3 + key4` - `key2 == key3`

### Імёны
- `name` IN `official_name` - у адным аб'екце
- `short_name` IN `name` - у адным аб'екце


### Адаміністратыўнае падзяленьне

- `boundary=administrative + admin_level + name` == `addr:region` - вобласьць
- `boundary=administrative + admin_level + name` == `addr:district` - раён
- `boundary=administrative + admin_level + name` == `addr:subdistrict` - сельскі савет

### Места

- `place + name` IN `name + boundary=administrative + admin_level` - вобласьць
- `place + name` IN `name + boundary=administrative + admin_level` - раён
- `place + name` IN `name + boundary=administrative + admin_level` - сельскі савет

- `place + name` == `addr:city` - места
- `place + name` == `addr:place` - места
- `place + name` == `addr:suburb`, `landuse + name` == `addr:suburb` - мікрараён

- `place + name` == `from` - маршрут
- `place + name` == `to` - маршрут
- `place + name` == `via` - маршрут
- `place + name` == `destination + highway` - маршрут
- `place + name` == `destination:backward + highway` - маршрут
- `place + name` == `destination:forward + highway` - маршрут

- `place + name` IN `official_name + highway` - дарога

- `place + name` == `fire_hydrant:city` - места

- `place + name` == `water_tank:city` - вадасхоішча

### Вуліца

- `highway + name` == `addr:street` - вуліца
- `highway + name` == `addr2:street` - вуліца

- `highway + name` == `from` - маршрут
- `highway + name` == `to` - маршрут
- `highway + name` == `via` - маршрут
- `highway + name` == `destination:street + highway` - маршрут
- `highway + name` == `destination:street:backward + highway` - маршрут
- `highway + name` == `destination:street:forward + highway` - маршрут

- `highway + name` == `fire_hydrant:street` - вуліца

### Іншае
- `waterway + name` == `destination + waterway` - рака